In [1]:
# Data Wrangling
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import seaborn as sns
%matplotlib inline

# EDA
# import klib

# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile
from sklearn import base
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.experimental import enable_iterative_imputer  # still experimental 
from sklearn.impute import IterativeImputer
from sklearn.feature_selection import RFE


# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVR
from lightgbm import LGBMClassifier
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.linear_model import BayesianRidge

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss

# Utility
import os
import time
import random
import sys, warnings
if not sys.warnoptions: warnings.simplefilter("ignore")
from IPython.display import Image
# import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean
from tensorflow import keras

# from bayes_opt import BayesianOptimization

# ver0로 실시

In [3]:
X_new = pd.read_csv(os.path.abspath("../input")+"/seo_X_new_selected.csv" , encoding = 'utf-8')
X_te_new = pd.read_csv(os.path.abspath("../input")+"/seo_X_te_new_selected.csv" , encoding = 'utf-8')

In [4]:
target = pd.read_csv(os.path.abspath("../input")+'/y_train.csv' , encoding = 'cp949').group

### *OOF 스태킹*

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
encoder = LabelEncoder()

In [7]:
y = encoder.fit_transform(target)

In [8]:
X_new.shape, X_te_new.shape

((21587, 540), (14380, 540))

In [9]:
lgb_oof_ver2 = np.zeros((X_new.shape[0], 19))
lgb_oof_ver2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from time import time
import datetime
import gc

# FE
from scipy.signal import find_peaks, peak_widths, peak_prominences

# Model
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
import xgboost as xgb

# Ensemble
from scipy.optimize import minimize
from sklearn.metrics import log_loss

In [14]:
clf = LGBMClassifier(
                objective='multiclass',
                metric='multi_logloss',
                n_estimators=round(361.6124537165089),
                learning_rate=0.03031787955469235,
                max_depth= round(10.543865435151808),
                max_bin = round(68.17658228757958),
                num_leaves= round(46.664058168263004),
                subsample= 0.5469702553792208,
                reg_alpha= 13.276820652062877,
                reg_lambda= 5.232957286613531,
                min_child_samples=round(153.5657486286675),
                min_child_weight= round(4.143227321090434),
                colsample_bytree = 0.5600982806065844,
                verbose=-1,
                silent=-1,
                random_state = 0
                )

In [34]:
n_splits = 10
mlogloss = []
lgb_oof_ver2 = np.zeros((X_new.shape[0], 8))
lgb_pred_ver2 = np.zeros((X_te_new.shape[0], 8))

for X, X_test in [(X_new,X_te_new)]:
    X= X.reset_index(drop=True)
    for seed in [0]:
        kfold = StratifiedKFold(n_splits=n_splits, random_state= seed, shuffle=True)
        for fold, (trn_idx, val_idx) in enumerate(kfold.split(X, y)):
            X_train, X_valid = X.iloc[trn_idx], X.iloc[val_idx]
            y_train, y_valid = y[trn_idx], y[val_idx]

            clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric= 'logloss', verbose= 200, 
                early_stopping_rounds= 200)
            # Predict
            lgb_pred_ver2 += clf.predict_proba(X_test) / (n_splits * 4)
            lgb_oof_ver2[val_idx] += clf.predict_proba(X_valid) / 4
            print('*'* 85)
            print('Training has finished.')
        print('lgb ver2 logloss= ', log_loss(y, lgb_oof_ver2))

[200]	training's multi_logloss: 0.98676	valid_1's multi_logloss: 1.48923
*************************************************************************************
Training has finished.
[200]	training's multi_logloss: 0.984068	valid_1's multi_logloss: 1.49922
*************************************************************************************
Training has finished.
[200]	training's multi_logloss: 0.985228	valid_1's multi_logloss: 1.49512
*************************************************************************************
Training has finished.
[200]	training's multi_logloss: 0.984598	valid_1's multi_logloss: 1.49482
*************************************************************************************
Training has finished.
[200]	training's multi_logloss: 0.98487	valid_1's multi_logloss: 1.47948
*************************************************************************************
Training has finished.
[200]	training's multi_logloss: 0.985398	valid_1's multi_logloss: 1.48226
*************

In [35]:
all_oof = np.column_stack([lgb_oof_ver2])
all_test = np.column_stack([lgb_pred_ver2])

In [36]:
X_train.shape

(19429, 540)

In [37]:
all_test.shape

(14380, 8)

In [38]:
mlogloss = []
n_splits = 10

stk_oof_pred = np.zeros((all_oof.shape[0], 8))
stk_test_pred = np.zeros((all_test.shape[0], 8))

kfold = StratifiedKFold(n_splits=n_splits, random_state=1000, shuffle=True)
for fold, (trn_idx, val_idx) in enumerate(kfold.split(all_oof, y)):
    X_train, X_valid = all_oof[trn_idx], all_oof[val_idx]
    y_train, y_valid = y[trn_idx], y[val_idx]
            
    clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric= 'logloss', verbose= 200, 
                early_stopping_rounds= 200)


    stk_test_pred += clf.predict_proba(all_test) / n_splits
    stk_oof_pred[val_idx] = clf.predict_proba(X_valid)
    
print('mean logloss= ',np.mean(mlogloss))

[200]	training's multi_logloss: 1.42136	valid_1's multi_logloss: 1.4744
[200]	training's multi_logloss: 1.42078	valid_1's multi_logloss: 1.47806
[200]	training's multi_logloss: 1.41826	valid_1's multi_logloss: 1.49747
[200]	training's multi_logloss: 1.42345	valid_1's multi_logloss: 1.45712
[200]	training's multi_logloss: 1.41804	valid_1's multi_logloss: 1.50733
[200]	training's multi_logloss: 1.41881	valid_1's multi_logloss: 1.49683
[200]	training's multi_logloss: 1.42158	valid_1's multi_logloss: 1.46486
[200]	training's multi_logloss: 1.42268	valid_1's multi_logloss: 1.46972
[200]	training's multi_logloss: 1.42074	valid_1's multi_logloss: 1.48357
[200]	training's multi_logloss: 1.41912	valid_1's multi_logloss: 1.48851
mean logloss=  nan


In [39]:
stk_test_pred = pd.DataFrame(stk_test_pred)

In [40]:
stk_test_pred = stk_test_pred.rename(columns = {0:'F20',1:'F30',2:'F40',3:'F50',4:'M20',5:'M30',6:'M40',7:'M50'})

In [41]:
stk_test_pred

,F20,F30,F40,F50,M20,M30,M40,M50
0,0.081727,0.126660,0.074350,0.163689,0.078011,0.288277,0.059040,0.128248
1,0.017167,0.286087,0.300252,0.018186,0.006559,0.067540,0.283459,0.020750
2,0.765935,0.171013,0.012716,0.010934,0.011603,0.015301,0.005153,0.007346
3,0.540564,0.124926,0.029789,0.047377,0.089036,0.126380,0.015090,0.026838
4,0.844976,0.077293,0.012056,0.009503,0.022793,0.016154,0.004160,0.013065
...,...,...,...,...,...,...,...,...
14375,0.115593,0.331190,0.029437,0.016352,0.015868,0.420053,0.061498,0.010009
14376,0.283721,0.336524,0.117023,0.056825,0.032202,0.087188,0.040652,0.045866
14377,0.406973,0.280906,0.078191,0.051105,0.053284,0.075907,0.028041,0.025593
14378,0.194520,0.411955,0.035614,0.015057,0.039477,0.250151,0.043620,0.009607


In [42]:
test = pd.read_csv(os.path.abspath("../input")+'/X_test.csv',encoding = 'cp949')
tst_id = test['custid'].unique()

In [43]:
submission = pd.concat([pd.DataFrame({'ID':tst_id}),stk_test_pred],axis = 1)
submission.to_csv('seo_lgbm_stk_oof_ver0.csv',index = False,encoding = 'utf-8')

In [44]:
submission

,ID,F20,F30,F40,F50,M20,M30,M40,M50
0,30001,0.081727,0.126660,0.074350,0.163689,0.078011,0.288277,0.059040,0.128248
1,30002,0.017167,0.286087,0.300252,0.018186,0.006559,0.067540,0.283459,0.020750
2,30003,0.765935,0.171013,0.012716,0.010934,0.011603,0.015301,0.005153,0.007346
3,30005,0.540564,0.124926,0.029789,0.047377,0.089036,0.126380,0.015090,0.026838
4,30007,0.844976,0.077293,0.012056,0.009503,0.022793,0.016154,0.004160,0.013065
...,...,...,...,...,...,...,...,...,...
14375,49988,0.115593,0.331190,0.029437,0.016352,0.015868,0.420053,0.061498,0.010009
14376,49990,0.283721,0.336524,0.117023,0.056825,0.032202,0.087188,0.040652,0.045866
14377,49992,0.406973,0.280906,0.078191,0.051105,0.053284,0.075907,0.028041,0.025593
14378,49993,0.194520,0.411955,0.035614,0.015057,0.039477,0.250151,0.043620,0.009607


# ver1로 실행

In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from time import time
import datetime
import gc

# FE
from scipy.signal import find_peaks, peak_widths, peak_prominences

# Model
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder

import lightgbm as lgb
import xgboost as xgb

# Ensemble
from scipy.optimize import minimize
from sklearn.metrics import log_loss

In [45]:
X_new = pd.read_csv(os.path.abspath("../input")+"/seo_X_new_selected_ver1.csv" , encoding = 'utf-8')
X_te_new = pd.read_csv(os.path.abspath("../input")+"/seo_X_te_new_selected_ver1.csv" , encoding = 'utf-8')

In [46]:
target = pd.read_csv(os.path.abspath("../input")+'/y_train.csv' , encoding = 'cp949').group

### *OOF 스태킹*

In [47]:
from sklearn.preprocessing import LabelEncoder

In [48]:
encoder = LabelEncoder()

In [49]:
y = encoder.fit_transform(target)

In [50]:
X_new.shape, X_te_new.shape

((21587, 561), (14380, 561))

In [51]:
lgb_oof_ver2 = np.zeros((X_new.shape[0], 19))
lgb_oof_ver2

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [52]:
clf = LGBMClassifier(
                objective='multiclass',
                metric='multi_logloss',
                n_estimators=round(402.299557563837),
                learning_rate=0.04345060405684092,
                max_depth= round(13.106545913327118),
                max_bin = round(121.4898573502621),
                num_leaves= round(42.29278213891062),
                subsample= 0.9255938277629163,
                reg_alpha= 19.792504424705974,
                reg_lambda= 7.942927734036579,
                min_child_samples=round(129.45048526924916),
                min_child_weight= round(41.8634794439083),
                colsample_bytree = 0.5451028501602284,
                verbose=-1,
                silent=-1,
                random_state = 0
                )

In [53]:
n_splits = 10
mlogloss = []
lgb_oof_ver2 = np.zeros((X_new.shape[0], 8))
lgb_pred_ver2 = np.zeros((X_te_new.shape[0], 8))

for X, X_test in [(X_new,X_te_new)]:
    X= X.reset_index(drop=True)
    for seed in [0]:
        kfold = StratifiedKFold(n_splits=n_splits, random_state= seed, shuffle=True)
        for fold, (trn_idx, val_idx) in enumerate(kfold.split(X, y)):
            X_train, X_valid = X.iloc[trn_idx], X.iloc[val_idx]
            y_train, y_valid = y[trn_idx], y[val_idx]

            clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric= 'logloss', verbose= 200, 
                early_stopping_rounds= 200)
            # Predict
            lgb_pred_ver2 += clf.predict_proba(X_test) / (n_splits * 4)
            lgb_oof_ver2[val_idx] += clf.predict_proba(X_valid) / 4
            print('*'* 85)
            print('Training has finished.')
        print('lgb ver2 logloss= ', log_loss(y, lgb_oof_ver2))

[200]	training's multi_logloss: 1.09025	valid_1's multi_logloss: 1.4839
[400]	training's multi_logloss: 0.859617	valid_1's multi_logloss: 1.47686
*************************************************************************************
Training has finished.
[200]	training's multi_logloss: 1.08852	valid_1's multi_logloss: 1.49091
[400]	training's multi_logloss: 0.860529	valid_1's multi_logloss: 1.48338
*************************************************************************************
Training has finished.
[200]	training's multi_logloss: 1.08951	valid_1's multi_logloss: 1.49047
[400]	training's multi_logloss: 0.858227	valid_1's multi_logloss: 1.48456
*************************************************************************************
Training has finished.
[200]	training's multi_logloss: 1.0892	valid_1's multi_logloss: 1.49581
[400]	training's multi_logloss: 0.859485	valid_1's multi_logloss: 1.48831
*************************************************************************************
T

In [54]:
all_oof = np.column_stack([lgb_oof_ver2])
all_test = np.column_stack([lgb_pred_ver2])

In [55]:
X_train.shape

(19429, 561)

In [56]:
all_test.shape

(14380, 8)

In [57]:
mlogloss = []
n_splits = 10

stk_oof_pred = np.zeros((all_oof.shape[0], 8))
stk_test_pred = np.zeros((all_test.shape[0], 8))

kfold = StratifiedKFold(n_splits=n_splits, random_state=1000, shuffle=True)
for fold, (trn_idx, val_idx) in enumerate(kfold.split(all_oof, y)):
    X_train, X_valid = all_oof[trn_idx], all_oof[val_idx]
    y_train, y_valid = y[trn_idx], y[val_idx]
            
    clf.fit(X_train, y_train, eval_set=[(X_train, y_train), (X_valid, y_valid)], eval_metric= 'logloss', verbose= 200, 
                early_stopping_rounds= 200)


    stk_test_pred += clf.predict_proba(all_test) / n_splits
    stk_oof_pred[val_idx] = clf.predict_proba(X_valid)
    
print('mean logloss= ',np.mean(mlogloss))

[200]	training's multi_logloss: 1.4294	valid_1's multi_logloss: 1.46904
[400]	training's multi_logloss: 1.42323	valid_1's multi_logloss: 1.46752
[200]	training's multi_logloss: 1.4291	valid_1's multi_logloss: 1.47407
[400]	training's multi_logloss: 1.4234	valid_1's multi_logloss: 1.47279
[200]	training's multi_logloss: 1.42675	valid_1's multi_logloss: 1.49676
[400]	training's multi_logloss: 1.4208	valid_1's multi_logloss: 1.49607
[200]	training's multi_logloss: 1.43078	valid_1's multi_logloss: 1.45444
[400]	training's multi_logloss: 1.42458	valid_1's multi_logloss: 1.45256
[200]	training's multi_logloss: 1.42639	valid_1's multi_logloss: 1.4973
[400]	training's multi_logloss: 1.42024	valid_1's multi_logloss: 1.4972
[200]	training's multi_logloss: 1.42969	valid_1's multi_logloss: 1.47952
[400]	training's multi_logloss: 1.42436	valid_1's multi_logloss: 1.47833
[200]	training's multi_logloss: 1.42964	valid_1's multi_logloss: 1.46043
[400]	training's multi_logloss: 1.42334	valid_1's multi_l

In [58]:
stk_test_pred = pd.DataFrame(stk_test_pred)

In [59]:
stk_test_pred = stk_test_pred.rename(columns = {0:'F20',1:'F30',2:'F40',3:'F50',4:'M20',5:'M30',6:'M40',7:'M50'})

In [60]:
stk_test_pred

,F20,F30,F40,F50,M20,M30,M40,M50
0,0.088096,0.136316,0.051363,0.115258,0.106704,0.307498,0.055745,0.139020
1,0.015979,0.257118,0.344955,0.027365,0.006075,0.038350,0.283133,0.027025
2,0.812149,0.113048,0.016937,0.010455,0.015427,0.017484,0.005720,0.008780
3,0.605275,0.133388,0.034212,0.036040,0.069613,0.084369,0.015888,0.021216
4,0.864990,0.052065,0.016110,0.010264,0.020261,0.017344,0.005458,0.013508
...,...,...,...,...,...,...,...,...
14375,0.173294,0.411145,0.026129,0.012413,0.021863,0.283142,0.060639,0.011374
14376,0.361832,0.267256,0.131235,0.046109,0.034261,0.086423,0.042813,0.030072
14377,0.475731,0.208004,0.074685,0.044025,0.059044,0.089802,0.023401,0.025308
14378,0.186320,0.444631,0.029680,0.010534,0.041584,0.245294,0.032537,0.009418


In [61]:
test = pd.read_csv(os.path.abspath("../input")+'/X_test.csv',encoding = 'cp949')
tst_id = test['custid'].unique()

In [62]:
submission = pd.concat([pd.DataFrame({'ID':tst_id}),stk_test_pred],axis = 1)
submission.to_csv('seo_lgbm_stk_oof_ver1.csv',index = False,encoding = 'utf-8')

In [63]:
submission

,ID,F20,F30,F40,F50,M20,M30,M40,M50
0,30001,0.088096,0.136316,0.051363,0.115258,0.106704,0.307498,0.055745,0.139020
1,30002,0.015979,0.257118,0.344955,0.027365,0.006075,0.038350,0.283133,0.027025
2,30003,0.812149,0.113048,0.016937,0.010455,0.015427,0.017484,0.005720,0.008780
3,30005,0.605275,0.133388,0.034212,0.036040,0.069613,0.084369,0.015888,0.021216
4,30007,0.864990,0.052065,0.016110,0.010264,0.020261,0.017344,0.005458,0.013508
...,...,...,...,...,...,...,...,...,...
14375,49988,0.173294,0.411145,0.026129,0.012413,0.021863,0.283142,0.060639,0.011374
14376,49990,0.361832,0.267256,0.131235,0.046109,0.034261,0.086423,0.042813,0.030072
14377,49992,0.475731,0.208004,0.074685,0.044025,0.059044,0.089802,0.023401,0.025308
14378,49993,0.186320,0.444631,0.029680,0.010534,0.041584,0.245294,0.032537,0.009418
